In [ ]:
import pandas as _hex_pandas
import datetime as _hex_datetime
import json as _hex_json

In [ ]:
hex_scheduled = _hex_json.loads("false")

In [ ]:
hex_user_email = _hex_json.loads("\"example-user@example.com\"")

In [ ]:
hex_user_attributes = _hex_json.loads("{}")

In [ ]:
hex_run_context = _hex_json.loads("\"logic\"")

In [ ]:
hex_timezone = _hex_json.loads("\"UTC\"")

In [ ]:
hex_project_id = _hex_json.loads("\"019620b0-962b-7000-8076-a5a6d29b5ddd\"")

In [ ]:
hex_project_name = _hex_json.loads("\"GENIUS UNSDG13 PUC-Rio\"")

In [ ]:
hex_status = _hex_json.loads("\"\"")

In [ ]:
hex_categories = _hex_json.loads("[]")

In [ ]:
hex_color_palette = _hex_json.loads("[\"#4C78A8\",\"#F58518\",\"#E45756\",\"#72B7B2\",\"#54A24B\",\"#EECA3B\",\"#B279A2\",\"#FF9DA6\",\"#9D755D\",\"#BAB0AC\"]")

In [ ]:
!pip install pandas faker gurobipy folium

Try `uv pip` instead for faster package installs:
!uv pip install pandas faker gurobipy folium
Learn more: https://learn.hex.tech/docs/explore-data/projects/environment-configuration/using-packages

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 126.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
import random
from faker import Faker
import math
import gurobipy as gp
from gurobipy import GRB
import folium
import webbrowser
import os
import pandas as pd
import datetime # For date handling if needed

In [ ]:
# Initialize Faker
fake = Faker()

# --- Constants and Defaults ---
DEFAULT_DEPRECIATION_YEARS = 10
DEFAULT_TAX_RATE = 0.3
DEFAULT_DISCOUNT_RATE = 0.10 # Annual discount rate for NPV
DEFAULT_T_MAX_MONTHS = 120 # 10 years horizon
MILLION = 1_000_000 # For display scaling
NPV_INVESTMENT_FACTOR = 1.1 # Portfolio NPV must be >= this factor * Portfolio Investment

In [ ]:
# --- 1. Generate Detailed Mock Data Function ---
def generate_detailed_project_data(num_projects, T_max, center_lat=-22.9, center_lon=-43.1):
    # ... (Function content from previous response - no changes needed here) ...
    project_data = {}
    time_periods = range(1, T_max + 1)
    print(f"Generating data for {num_projects} projects (Units: Abs $, MWh, Ton, Area km^2)...")
    for i in range(1, num_projects + 1):
        project_id = f"Proj_{i:03d}"
        area_needed = round(random.uniform(0.1, 50.0), 3); jobs_created = random.randint(5, 100)
        local_content_pct = round(random.uniform(0.1, 0.9), 2); hdi_location = round(random.uniform(0.5, 0.9), 3)
        latitude = round(center_lat + random.uniform(-1.5, 1.5), 6); longitude = round(center_lon + random.uniform(-1.5, 1.5), 6)
        time_to_complete = random.randint(12, T_max - 6)
        base_investment = round(random.uniform(1*MILLION, 100*MILLION), 2); base_energy_price = random.uniform(40, 150)
        base_energy_output = random.uniform(100, 2000); base_fixed_op_cost = base_investment * random.uniform(0.005, 0.02) / 12
        base_var_op_cost_pu = base_energy_price * random.uniform(0.1, 0.4); base_carbon_emissions_pu = random.uniform(0.0, 0.6)
        base_carbon_cost = random.uniform(20, 100); base_decommissioning = base_investment * random.uniform(0.05, 0.15)
        investment_ts = {}; energy_price_ts = {}; energy_output_ts = {}; fixed_op_costs_ts = {}; variable_op_costs_ts = {}
        decommissioning_ts = {}; carbon_emissions_ts = {}; carbon_cost_ts = {}; carbon_inventory_ts = {}; revenue_per_month = {}
        investment_per_period = base_investment / time_to_complete if time_to_complete > 0 else 0
        decommissioning_per_period = base_decommissioning / 24 if T_max >= 24 else 0
        total_operating_cost = 0; operating_months = 0
        for t in time_periods:
            investment_ts[t] = investment_per_period * (1 + random.uniform(-0.1, 0.1)) if t <= time_to_complete else 0
            energy_price_ts[t] = round(max(10, base_energy_price + (t / T_max * base_energy_price * 0.2) + random.uniform(-5, 5)), 2)
            eo_t = base_energy_output * (1 + random.uniform(-0.05, 0.05)) if t > time_to_complete else 0
            energy_output_ts[t] = round(max(0, eo_t), 2)
            foc_t = base_fixed_op_cost * (1 + (t/T_max)*0.1 + random.uniform(-0.05, 0.05))
            fixed_op_costs_ts[t] = round(max(0, foc_t), 2)
            voc_pu = base_var_op_cost_pu * (1 + (t/T_max)*0.05 + random.uniform(-0.05, 0.05))
            variable_op_costs_ts[t] = round(max(0, energy_output_ts[t] * voc_pu), 2)
            decom_t = decommissioning_per_period if t > (T_max - 24) else 0
            decommissioning_ts[t] = decom_t
            ce_pu = base_carbon_emissions_pu * (1 + random.uniform(-0.05, 0.05))
            carbon_emissions_ts[t] = round(max(0, energy_output_ts[t] * ce_pu), 4)
            cc_t = base_carbon_cost * (1 + (t/T_max)*0.3 + random.uniform(-3, 3))
            carbon_cost_ts[t] = round(max(5, cc_t), 2)
            carbon_inventory_ts[t] = random.uniform(-base_energy_output*0.05, base_energy_output*0.05)
            if t > time_to_complete:
                 carbon_cost_total_t = carbon_emissions_ts[t] * carbon_cost_ts[t]
                 total_operating_cost += fixed_op_costs_ts[t] + variable_op_costs_ts[t] + carbon_cost_total_t + decom_t
                 operating_months += 1
        target_profit = base_investment * random.uniform(0.1, 0.5) # Ensure positive profit vs II
        # Make required revenue slightly higher to increase chance of meeting NPV >= 1.1*II
        required_total_revenue = total_operating_cost + base_investment * (5*NPV_INVESTMENT_FACTOR + random.uniform(0.05, 0.3)) # Ensure Rev > OpEx + 1.1*II + margin
        if operating_months > 0:
            avg_req_revenue = required_total_revenue / operating_months
            for t in time_periods:
                if t > time_to_complete: revenue_per_month[t] = round(max(0, avg_req_revenue * (1 + random.uniform(0.0, 0.3))), 2)
                else: revenue_per_month[t] = 0
        else:
             for t in time_periods: revenue_per_month[t] = 0
        project_data[project_id] = {
            'Area needed': area_needed, 'Jobs created': jobs_created, 'Local content (%)': local_content_pct,
            'HDI from location': hdi_location, 'TC': time_to_complete, 'latitude': latitude, 'longitude': longitude,
            'Investment': investment_ts, 'Energy price': energy_price_ts, 'Energy output': energy_output_ts,
            'Fixed operating costs': fixed_op_costs_ts, 'Variable operating costs': variable_op_costs_ts,
            'Decommissioning': decommissioning_ts, 'Carbon emissions': carbon_emissions_ts,
            'Carbon cost': carbon_cost_ts, 'Carbon inventory': carbon_inventory_ts, 'Revenue': revenue_per_month
        }
    print("Data generation complete.")
    return project_data


# --- 2. Excel Loading Function ---
def load_data_from_excel(excel_filepath, T_max):
    # ... (Function content from previous response - no changes needed here) ...
    print(f"\nLoading data from Excel file: {excel_filepath} (Units: Abs $, MWh, Ton, Area km^2 expected)...")
    project_data = {}; non_ts_vars = ['Area needed', 'Jobs created', 'Local content (%)', 'HDI from location', 'TC']
    expected_ts_vars = [ 'Investment', 'Energy price', 'Energy output', 'Fixed operating costs', 'Variable operating costs', 'Decommissioning', 'Carbon emissions', 'Carbon cost', 'Carbon inventory', 'Revenue' ]
    expected_vars = expected_ts_vars + non_ts_vars
    try:
        xls = pd.ExcelFile(excel_filepath); sheet_names = xls.sheet_names; print(f"Found sheets (projects): {sheet_names}")
        for sheet_name in sheet_names:
            proj_id = sheet_name; print(f"  Processing project: {proj_id}..."); proj_dict = {}
            try:
                df = pd.read_excel(excel_filepath, sheet_name=sheet_name, index_col=0, header=0)
                original_headers = df.columns.tolist(); parsed_dates = {}
                for header in original_headers:
                    try: parsed = pd.to_datetime(header, errors='coerce', format=None)
                    except Exception: parsed = pd.NaT
                    if pd.notna(parsed): parsed_dates[header] = parsed
                    else: print(f"    Warning: Could not parse header '{header}'. Skipping.")
                if not parsed_dates: print(f"    ERROR: No valid date headers found. Skipping sheet."); continue
                sorted_headers = sorted(parsed_dates.keys(), key=lambda h: parsed_dates[h])
                header_to_t_map = {header: t + 1 for t, header in enumerate(sorted_headers)}; effective_T_max = len(sorted_headers)
                if effective_T_max == 0 : print(f"    ERROR: Zero valid date columns found. Skipping."); continue
                if effective_T_max < T_max: print(f"    Warning: Found {effective_T_max} valid date columns < T_max ({T_max}). Padding.")
                elif effective_T_max > T_max: print(f"    Warning: Found {effective_T_max} valid date columns > T_max ({T_max}). Using first {T_max}."); sorted_headers = sorted_headers[:T_max]; header_to_t_map = {h: t+1 for t, h in enumerate(sorted_headers)}; effective_T_max = T_max
            except Exception as read_err: print(f"    ERROR: Could not read/parse sheet '{proj_id}'. Skipping. Error: {read_err}"); continue
            first_period_header = sorted_headers[0]
            for var_name in expected_vars:
                if var_name not in df.index: print(f"    Warning: Variable '{var_name}' not found. Setting default/empty."); proj_dict[var_name] = 0 if var_name in non_ts_vars else {t: 0.0 for t in range(1, T_max + 1)}; continue
                if var_name in non_ts_vars:
                    try: value = df.loc[var_name, first_period_header]; value = 0 if pd.isna(value) else value; proj_dict[var_name] = int(value) if var_name in ['Jobs created', 'TC'] else float(value)
                    except (KeyError, ValueError): print(f"    Warning: Issue getting/converting non-TS var '{var_name}'. Using 0."); proj_dict[var_name] = 0 if var_name in ['Jobs created', 'TC'] else 0.0
                else: # Time-series
                    ts_dict = {};
                    try:
                        ts_row = df.loc[var_name]
                        for header in sorted_headers: t = header_to_t_map[header]; value = ts_row.get(header, 0.0); ts_dict[t] = 0.0 if pd.isna(value) else float(value)
                        for t_pad in range(effective_T_max + 1, T_max + 1): ts_dict[t_pad] = 0.0
                        proj_dict[var_name] = ts_dict
                    except Exception as ts_err: print(f"    Warning: Error processing TS for '{var_name}'. Using empty. Error: {ts_err}"); proj_dict[var_name] = {t: 0.0 for t in range(1, T_max + 1)}
            project_data[proj_id] = proj_dict
        print(f"Data loaded successfully for {len(project_data)} projects from Excel.")
        return project_data
    except FileNotFoundError: print(f"ERROR: Excel file not found at {excel_filepath}"); return None
    except Exception as e: print(f"ERROR loading data from Excel: {e}"); return None

# --- 3. Calculate Metrics Function ---
def calculate_project_metrics_v2(project_data, T_max, tax_rate, discount_rate_annual, depreciation_years):
    # ... (Function content from previous response - no changes needed here) ...
    results = {}
    discount_rate_monthly = (1 + discount_rate_annual)**(1/12) - 1
    depreciation_periods = depreciation_years * 12
    print("\nCalculating derived metrics, NPV, and Totals (Units: Abs $, MWh, Ton, Area km^2)...")
    for proj_id, data in project_data.items():
        period_profit_ts = {}; depreciation_ts = {}; cash_flow_ts = {}; revenue_per_unit_energy = {}; cost_per_unit_energy = {}; profit_per_unit_energy = {}; area_per_unit_energy = {}
        total_emissions = 0; total_energy_output = 0
        TC = data.get('TC', T_max); investment_dict = data.get('Investment', {}); energy_output_dict = data.get('Energy output', {})
        energy_price_dict = data.get('Energy price', {}); fixed_op_costs_dict = data.get('Fixed operating costs', {}); variable_op_costs_dict = data.get('Variable operating costs', {})
        carbon_emissions_dict = data.get('Carbon emissions', {}); carbon_cost_dict = data.get('Carbon cost', {}); decommissioning_dict = data.get('Decommissioning', {})
        revenue_dict = data.get('Revenue', {})
        area_needed = data.get('Area needed', 0); initial_investment = sum(investment_dict.get(t, 0) for t in range(1, TC + 1))
        jobs_created = data.get('Jobs created', 0); hdi_location = data.get('HDI from location', 1.0)
        depreciation_base_value = initial_investment / depreciation_periods if depreciation_periods > 0 and initial_investment > 0 else 0 # Avoid division by zero
        for t in range(1, T_max + 1):
            invest_t = investment_dict.get(t, 0); eo_t = energy_output_dict.get(t, 0); rev_t = revenue_dict.get(t,0)
            foc_t = fixed_op_costs_dict.get(t, 0); voc_t = variable_op_costs_dict.get(t, 0); ce_t = carbon_emissions_dict.get(t, 0)
            cc_t = carbon_cost_dict.get(t, 0); decom_t = decommissioning_dict.get(t, 0)
            if t > TC: total_emissions += ce_t; total_energy_output += eo_t
            depr_t = 0
            if t > TC and (t - TC) <= depreciation_periods: depr_t = depreciation_base_value
            depreciation_ts[t] = depr_t
            carbon_cost_total_t = ce_t * cc_t
            taxable_profit_t = rev_t - foc_t - voc_t - depr_t
            tax_t = max(0, taxable_profit_t * tax_rate)
            profit_t = rev_t - (foc_t + voc_t + carbon_cost_total_t) - tax_t - decom_t
            period_profit_ts[t] = profit_t; cash_flow_ts[t] = profit_t + depr_t - invest_t
            if eo_t > 1e-6:
                revenue_per_unit_energy[t] = rev_t / eo_t; cost_per_unit_energy[t] = (foc_t + voc_t + carbon_cost_total_t + decom_t) / eo_t
                profit_per_unit_energy[t] = profit_t / eo_t; area_per_unit_energy[t] = area_needed / eo_t
            else: revenue_per_unit_energy[t]=0; cost_per_unit_energy[t]=0; profit_per_unit_energy[t]=0; area_per_unit_energy[t]=float('inf') if area_needed>0 else 0
        npv = sum(cash_flow_ts.get(t, 0) / ((1 + discount_rate_monthly) ** t) for t in range(1, T_max + 1))
        avg_emissions_per_unit = total_emissions / total_energy_output if total_energy_output > 1e-6 else float('inf')
        results[proj_id] = {
            'NPV': npv, 'II': initial_investment, 'Jobs': jobs_created, 'HDI': hdi_location, 'TotalEmissions': total_emissions,
            'TotalEnergyOutput': total_energy_output, 'AvgEmissionsPerUnit': avg_emissions_per_unit, 'TC': TC,
            'Area needed': area_needed, 'Local content (%)': data.get('Local content (%)'),
        }
    print("Calculations complete.")
    return results


# --- 4. Solver Function (Weighted Sum - MODIFIED NPV CONSTRAINT) ---
def solve_weighted_sum(metrics, TB, W_npv, W_emissions, W_jobs, W_hdi, npv_invest_factor):
    """
    Solves the multi-objective problem using a weighted sum.
    Includes constraint for Portfolio NPV >= npv_invest_factor * Portfolio Investment.
    Weights MUST be scaled appropriately.
    """
    projects = list(metrics.keys())
    m = gp.Model("PortfolioWeightedSumNPVFactor")
    m.setParam('OutputFlag', 0)
    x = m.addVars(projects, vtype=GRB.BINARY, name="Select")

    # Objective Components
    obj_npv = gp.quicksum(x[i] * metrics[i]['NPV'] for i in projects)
    obj_emissions = gp.quicksum(x[i] * metrics[i]['TotalEmissions'] for i in projects)
    obj_jobs = gp.quicksum(x[i] * metrics[i]['Jobs'] for i in projects)
    obj_hdi_priority = gp.quicksum(x[i] * (1.0 - metrics[i].get('HDI', 1.0)) for i in projects)
    # Portfolio Investment Cost component (used in constraint)
    portfolio_investment = gp.quicksum(x[i] * metrics[i]['II'] for i in projects)


    # Combined Weighted Objective
    m.setObjective(
        W_npv * obj_npv
        - W_emissions * obj_emissions
        + W_jobs * obj_jobs
        + W_hdi * obj_hdi_priority,
        GRB.MAXIMIZE
    )

    # --- Constraints ---
    # Budget Constraint
    m.addConstr(portfolio_investment <= TB, "Budget")

    # ***** MODIFIED CONSTRAINT: Portfolio NPV >= factor * Portfolio Investment *****
    #m.addConstr(obj_npv >= npv_invest_factor * portfolio_investment, "PortfolioNPV_vs_Investment")
    # ***** END MODIFIED CONSTRAINT *****

    m.optimize()

    # --- Extract results ---
    selected_details = []; objective_value = None; is_optimal = False
    actual_metrics = {'NPV': 0, 'Emissions': 0, 'Jobs': 0, 'Cost': 0, 'AvgHDI': 0, 'Count': 0}

    if m.Status == GRB.OPTIMAL or m.Status == GRB.SUBOPTIMAL:
        # Check constraint
        portfolio_npv_check = obj_npv.getValue()
        portfolio_inv_check = portfolio_investment.getValue()
        if portfolio_npv_check < npv_invest_factor * portfolio_inv_check - 1e-6: # Check with tolerance
             print(f"WARNING: Optimal solution reported but NPV constraint may be violated?")
             print(f"  NPV: {portfolio_npv_check:,.2f}, Required: {npv_invest_factor * portfolio_inv_check:,.2f}")

        objective_value = m.ObjVal; is_optimal = True; selected_count = 0; total_hdi_sum = 0
        for i in projects:
            if x[i].X > 0.5:
                 cost = metrics[i]['II']; selected_details.append({'id': i, 'cost': cost})
                 actual_metrics['NPV'] += metrics[i]['NPV']; actual_metrics['Emissions'] += metrics[i]['TotalEmissions']
                 actual_metrics['Jobs'] += metrics[i]['Jobs']; actual_metrics['Cost'] += cost
                 total_hdi_sum += metrics[i].get('HDI', 1.0); selected_count +=1
        actual_metrics['Count'] = selected_count
        if selected_count > 0: actual_metrics['AvgHDI'] = total_hdi_sum / selected_count

    elif m.Status == GRB.INFEASIBLE:
         print(f"Model is infeasible (No solution satisfies Budget and NPV >= {npv_invest_factor}*Investment constraints).")
         return [], None, False, None # Indicate infeasibility
    else:
         print(f"Optimization finished with status: {m.Status}")
         return [], None, False, None

    return selected_details, objective_value, is_optimal, actual_metrics


# --- 5. Map Creation Function ---
def create_project_map(project_data_for_locations, selected_projects_details, filename="project_map_final.html"):
    # ... (Function content remains the same) ...
    print(f"\nCreating map: {filename}...")
    if not project_data_for_locations: return None, None
    try:
        valid_ids = set(project_data_for_locations.keys())
        filtered_details = [p for p in selected_projects_details if p['id'] in valid_ids]
        selected_ids = {p['id'] for p in filtered_details}
        all_lats = [p['latitude'] for p in project_data_for_locations.values() if 'latitude' in p]; all_lons = [p['longitude'] for p in project_data_for_locations.values() if 'longitude' in p]
        if not all_lats or not all_lons: print("No location data found."); return None, None
        avg_lat = sum(all_lats) / len(all_lats); avg_lon = sum(all_lons) / len(all_lons)
        map_object = folium.Map(location=[avg_lat, avg_lon], zoom_start=8)
        for proj_id, data in project_data_for_locations.items():
             if 'latitude' not in data or 'longitude' not in data: continue
             ii_cost = data.get('II', sum(data.get('Investment', {}).values()))
             cost_display = f"${ii_cost / MILLION:,.1f} M" if ii_cost > 0 else "$0.0 M"
             popup_text = f"ID: {proj_id}<br>Cost: {cost_display}"
             if proj_id in selected_ids:
                  sel_cost = next((item['cost'] for item in filtered_details if item['id'] == proj_id), ii_cost)
                  sel_cost_display = f"${sel_cost / MILLION:,.1f} M" if sel_cost > 0 else "$0.0 M"
                  folium.Marker( location=[data['latitude'], data['longitude']], popup=f"<b>Selected: {proj_id}</b><br>Cost: {sel_cost_display}", icon=folium.Icon(color='red', icon='industry', prefix='fa') ).add_to(map_object)
             else:
                  folium.CircleMarker( location=[data['latitude'], data['longitude']], radius=3, color='grey', fill=True, fill_color='grey', fill_opacity=0.6, popup=popup_text ).add_to(map_object)
        filepath = os.path.abspath(filename); map_object.save(filepath)
        print(f"Map saved successfully to: {filepath}")
        return filepath, map_object
    except Exception as e: print(f"Error creating or saving map: {e}"); return None, None

In [ ]:
# ==============================================
# --- Main Execution Script ---
# ==============================================
if __name__ == "__main__":

    # --- 1. Define Parameters ---
    T_max_months_run = DEFAULT_T_MAX_MONTHS
    Total_Budget_Run = 500 * MILLION # 500 M$ Budget (absolute $)
    Num_Projects_Run = 500
    NPV_Investment_Factor_Run = NPV_INVESTMENT_FACTOR # Use the constant defined earlier (1.1)

    # --- Multi-Objective Weights (SCALING IS CRITICAL) ---
    W_npv_run = 1.0
    W_emissions_run = 100
    W_jobs_run = 100000
    W_hdi_run = 50 * MILLION
    print("\n--- Optimization Weights (MUST BE SCALED) ---")
    print(f"W_npv: {W_npv_run} (per $)"); print(f"W_emissions: {W_emissions_run} (penalty per Ton)")
    print(f"W_jobs: {W_jobs_run} (value per Job)"); print(f"W_hdi: {W_hdi_run} (value per unit of (1-HDI) score)")
    print("-" * 40)

    # Calculation Parameters
    tax_rate_run = DEFAULT_TAX_RATE; discount_rate_run = DEFAULT_DISCOUNT_RATE; depreciation_years_run = DEFAULT_DEPRECIATION_YEARS

    # --- 2. Load or Generate Data ---
    project_data_raw = generate_detailed_project_data(Num_Projects_Run, T_max_months_run)
    # OR: project_data_raw = load_data_from_excel("your_file.xlsx", T_max_months_run)

    # --- 3. Calculate Metrics ---
    metrics = None
    if project_data_raw: metrics = calculate_project_metrics_v2( project_data_raw, T_max_months_run, tax_rate_run, discount_rate_run, depreciation_years_run )

    # ***** NEW: Check individual project viability *****
    if metrics:
        print("\n--- Individual Project NPV vs Investment Check ---")
        viable_projects_count = 0
        for proj_id, data in metrics.items():
            npv = data['NPV']
            invest = data['II']
            required_npv = NPV_Investment_Factor_Run * invest
            if invest > 0 and npv >= required_npv:
                print(f"  Project {proj_id}: VIABLE (NPV ${npv/MILLION:.2f}M >= Required ${required_npv/MILLION:.2f}M)")
                viable_projects_count += 1
        # Optional: Print non-viable ones too for comparison
            # else:
        #       print(f"  Project {proj_id}: Non-Viable (NPV ${npv/MILLION:.2f}M < Required ${required_npv/MILLION:.2f}M)")
        print(f"Found {viable_projects_count} projects potentially meeting the NPV >= {NPV_Investment_Factor_Run}*Investment constraint individually.")
    if viable_projects_count == 0:
        print("WARNING: No single project meets the strict NPV constraint. Infeasibility is expected.")
    print("-" * 40)
# ***** END Check *****

    # --- 4. Solve using Weighted Sum ---
    selected_details, objective, is_optimal, actual_metrics_dict = [], None, False, None
    if metrics:
        if metrics:
             selected_details, objective, is_optimal, actual_metrics_dict = solve_weighted_sum(
                 metrics=metrics, TB=Total_Budget_Run, W_npv=W_npv_run,
                 W_emissions=W_emissions_run, W_jobs=W_jobs_run, W_hdi=W_hdi_run,
                 npv_invest_factor=NPV_Investment_Factor_Run # Pass the factor
             )
        else: print("Metrics dictionary is empty. Skipping optimization.")
    else: print("Skipping optimization as metrics could not be calculated.")

    # --- 5. Print Summary ---
    print("\n--- Run Summary ---")
    if actual_metrics_dict is not None and objective is not None and is_optimal:
        print(f"Optimization Status: Optimal/Suboptimal Solution Found (Weighted Sum, NPV >= {NPV_Investment_Factor_Run}*Invest)") # Updated status
        print(f"Combined Weighted Objective Value: {objective:,.2f}")
        print("\nSelected Projects Details (ID, Initial Cost):")
        if selected_details:
             print(f"  Count: {actual_metrics_dict['Count']}")
             for proj in selected_details: print(f"  - ID: {proj['id']}, Cost: ${proj['cost']/MILLION:,.2f} M")
             print(f"\nPortfolio Performance Metrics:")
             print(f"  Total Initial Cost: ${actual_metrics_dict['Cost']/MILLION:,.2f} M")
             print(f"  Budget Utilization: {(actual_metrics_dict['Cost'] / Total_Budget_Run) * 100:.2f}%")
             print(f"  Total NPV: ${actual_metrics_dict['NPV']/MILLION:,.2f} M")
             # Verify NPV vs Investment constraint met
             npv_check = actual_metrics_dict['NPV']
             cost_check = actual_metrics_dict['Cost']
             required_npv = NPV_Investment_Factor_Run * cost_check
             print(f"  NPV vs Required ({NPV_Investment_Factor_Run}*Cost): ${npv_check/MILLION:,.2f} M >= ${required_npv/MILLION:,.2f} M -> {'Met' if npv_check >= required_npv - 1e-6 else 'VIOLATED (check solver)'}")
             print(f"  Total Emissions: {actual_metrics_dict['Emissions']:,.2f} Tons")
             print(f"  Total Jobs Created: {actual_metrics_dict['Jobs']:.0f}")
             print(f"  Average HDI of Selected: {actual_metrics_dict['AvgHDI']:.3f}")
        else: print("  (No projects selected meeting all constraints)")
    elif metrics is not None and not is_optimal and objective is None:
         print(f"Optimization Status: Model Infeasible (No solution satisfies Budget and NPV >= {NPV_Investment_Factor_Run}*Investment constraints).")
    elif metrics is None: print("Optimization Status: Could not calculate project metrics.")
    else: print("Optimization Status: Solver failed or did not find an optimal/suboptimal solution.")

    # --- 6. Create and Optionally Open Map ---
    if project_data_raw:
        current_selection = selected_details if selected_details is not None else []
        map_file_path, map_obj = create_project_map(project_data_raw, current_selection)
        if map_file_path:
            try: webbrowser.open(f"file://{map_file_path}")
            except Exception as e: print(f"Could not automatically open map file: {e}")
        # if map_obj: map_obj
map_obj


--- Optimization Weights (MUST BE SCALED) ---
W_npv: 1.0 (per $)
W_emissions: 100 (penalty per Ton)
W_jobs: 100000 (value per Job)
W_hdi: 50000000 (value per unit of (1-HDI) score)
----------------------------------------
Generating data for 500 projects (Units: Abs $, MWh, Ton, Area km^2)...
Data generation complete.

Calculating derived metrics, NPV, and Totals (Units: Abs $, MWh, Ton, Area km^2)...
Calculations complete.

--- Individual Project NPV vs Investment Check ---
  Project Proj_001: VIABLE (NPV $184.20M >= Required $89.78M)
  Project Proj_002: VIABLE (NPV $28.28M >= Required $13.68M)
  Project Proj_003: VIABLE (NPV $51.03M >= Required $30.32M)
  Project Proj_004: VIABLE (NPV $7.33M >= Required $3.80M)
  Project Proj_005: VIABLE (NPV $110.72M >= Required $103.31M)
  Project Proj_006: VIABLE (NPV $93.62M >= Required $61.17M)
  Project Proj_007: VIABLE (NPV $83.95M >= Required $77.15M)
  Project Proj_008: VIABLE (NPV $33.88M >= Required $28.51M)
  Project Proj_009: VIABLE (NP

<!DOCTYPE html>

In [ ]:
import json as _hex_json

project_data = None

In [ ]:
map_obj

<!DOCTYPE html>